In [0]:
%sql
MERGE INTO flights_gold.gold_flights_f AS target
USING(
 SELECT
    CONCAT(f.FLIGHT_NUMBER,'-',f.AIRLINE,'-',f.TAIL_NUMBER) AS FLIGHT_ID,
    a.airline_key AS AIRLINE_KEY,
    orig.airport_key AS ORIG_AIRPORT_KEY,
    dest.airport_key AS DEST_AIRPORT_KEY,
 
    d.date_key AS DATE_KEY,
    
    f.DEPARTURE_DELAY,

    f.ARRIVAL_DELAY,
    f.SECURITY_DELAY,
    f.AIRLINE_DELAY,
    f.LATE_AIRCRAFT_DELAY,
    f.WEATHER_DELAY,
    
    f.AIR_TIME,
    f.ELAPSED_TIME,
    f.DISTANCE,
    
    f.CANCELLED,
    f.DIVERTED
FROM flight_silver.fact_flights f
LEFT JOIN flight_silver.dim_airline a
    ON TRIM(f.AIRLINE) = TRIM(a.IATA_CODE) AND a.is_current = TRUE
LEFT JOIN flight_silver.dim_airport orig
    ON TRIM(f.ORIGIN_AIRPORT) = TRIM(orig.IATA_CODE) AND orig.is_current = TRUE
LEFT JOIN flight_silver.dim_airport dest
    ON TRIM(f.DESTINATION_AIRPORT) = TRIM(dest.IATA_CODE) AND dest.is_current = TRUE
LEFT JOIN flight_silver.dim_date d
    ON f.YEAR = d.year
    AND f.MONTH = d.month
    AND f.DAY = d.day   
) AS source
ON target.FLIGHT_ID = source.FLIGHT_ID
WHEN NOT MATCHED THEN INSERT *;

SELECT * FROM flights_gold.gold_flights_f;